# Preparando el entorno: 
* Importación de librerías
* Abriendo Google Drive

In [0]:
%tensorflow_version 2.x 

import matplotlib.pyplot as plt
import tensorflow as tf
import re
import numpy as np
import json
from glob import glob
from PIL import Image
import pickle
import keras
import sys, time, os, warnings 
import numpy as np
import pandas as pd 
from collections import Counter 

from sklearn.model_selection import train_test_split
from sklearn.utils import shuffle


from google.colab import drive
drive.mount('/content/drive')

Estableciendo las rutas para acceder a las imágenes y a las descripciones.

In [0]:
# La ruta al directorio con las imagenes del dataset
dir_Flickr_jpg = '/content/drive/My Drive/Colab Notebooks/ImageCaptioning/Flickr8k/Flickr8k_Dataset/'

# La ruta al archivo con las descripciones
dir_Flickr_text = '/content/drive/My Drive/Colab Notebooks/ImageCaptioning/Flickr8k/Flickr8k_texto/es_Flickr8k.token.txt'

jpgs = os.listdir(dir_Flickr_jpg)
print("Cantidad de imagenes en Flicker8k: {}".format(len(jpgs)))

# Análisis preliminar de los datos:
* Guardando las descripciones y los IDs en un pandas dataframe

In [0]:
# Leyendo las descripciones
file = open(dir_Flickr_text, 'r')
text = file.read()
file.close()

# Extrayendo la información y guardando en el dataframe
lineas_txt = list()
for line in text.split('\n'): # Separamos por líneas
  column = line.split(',') # Separamos por columnas
  if len(column) == 1:
    continue
  w = [column[0], column[1], column[2]]
  lineas_txt.append(w)

captions_df = pd.DataFrame(lineas_txt,columns=['filename','caption_index','caption_text'])
print(captions_df.head())

# Obteniendo la información de las descripciones
uni_filenames = np.unique(captions_df.filename.values)
print("Total de imgaénes: : {}".format(len(uni_filenames)))
print("Cantidad de descripciones por imagen:")
Counter(Counter(captions_df.filename.values).values())

Se observarán las descripciones a la par de la imagen que describen según el dataset.

In [0]:
from keras.preprocessing.image import load_img, img_to_array

npic = 5
npix = 224
target_size = (npix,npix,3)

count = 1
fig = plt.figure(figsize=(10,20))
for jpgfnm in uni_filenames[:npic]:
    filename = dir_Flickr_jpg + '/' + jpgfnm
    captions = list(captions_df["caption_text"].loc[captions_df["filename"]==jpgfnm].values)
    image_load = load_img(filename, target_size=target_size)
    
    ax = fig.add_subplot(npic,2,count,xticks=[],yticks=[])
    ax.imshow(image_load)
    count += 1
    
    ax = fig.add_subplot(npic,2,count)
    plt.axis('off')
    ax.plot()
    ax.set_xlim(0,1)
    ax.set_ylim(0,len(captions))
    for i, caption in enumerate(captions):
        ax.text(0,i,caption,fontsize=20)
    count += 1
plt.show()

# Análisis de las descripciones


Se creará un dataframe para visualizar la distribución de las palabras en nuestro conjunto de datos traducido, este nuevo dataframe contendrá cada palabra y su respectiva frecuencia en orden descendente.

In [0]:
def df_word(df_txt):
  vocabulary = []
  for txt in df_txt.caption_text.values:
    vocabulary.extend(txt.split())
  print('Tamaño del vocabulario: %d' % len(set(vocabulary)))
  ct = Counter(vocabulary)
  dfword = pd.DataFrame.from_dict(ct, orient='index').sort_values(by=0, ascending=False)
  dfword = dfword.reset_index()
  dfword = dfword.rename({'index':'word',0:'count'}, axis=1)
  return(dfword)

dfword = df_word(captions_df)
dfword.head()

Se visualizan las palabras más y menos frecuentes que aparecen en todo el vocabulario.

Nota: Se puede observar que la mayoría de palabras en el vocabulario son articulos que no aportan mucha información sobre los datos.

In [0]:
top = 50

def plthist(dfsub, title):
    plt.figure(figsize=(20,3))
    plt.bar(dfsub.index,dfsub["count"])
    plt.yticks(fontsize=20)
    plt.xticks(dfsub.index,dfsub["word"],rotation=90,fontsize=20)
    plt.title(title,fontsize=20)
    plt.show()

plthist(dfword.iloc[:top,:], title="Las 50 palabras más frecuentes")
plthist(dfword.iloc[-top:,:], title="Las 50 palabras menos frecuentes")

# Preparación de los datos

* Se preparan los datos de forma separada a las imágenes
* Se desarrollará el modelo con "Visual Attention"




In [0]:
from sklearn.utils import shuffle

# Obteniendo todas las rutas a las imágenes
all_image_names = captions_df['filename'].tolist()

all_captions_n = captions_df['caption_text'].tolist()

all_img_name_vector = list()
for name in all_image_names:
  full_path = dir_Flickr_jpg + name
  all_img_name_vector.append(full_path)

all_captions = list()
for caption in all_captions_n:
  new_caption = '<start> ' + caption + ' <end>'
  all_captions.append(new_caption)

# Se hace un random para obtener el conjunto de entrenamiento
train_captions_, img_name_vector_ = shuffle(all_captions,
                                          all_img_name_vector,
                                          random_state=1)



# Se utilizarán 8000 (de 8092) ejemplos para entrenamiento
num_examples = 35900 # Cantidad de descripciones
# Set de entrenamiento
train_captions = train_captions_[:num_examples]
img_name_vector = img_name_vector_[:num_examples]

print("img_name_vector", img_name_vector[:10])
print("train_captions", train_captions[:10])

# Set de prueba
test_captions = train_captions_[num_examples:]
fnm_test = img_name_vector_[num_examples:]

print("Entrenamiento con: ",len(train_captions), "descripciones, de un total de:", len(all_captions))

# Preprocesando las imágenes con InceptionV3

Se utilizará InceptionV3 (que está preentrenada sobre Imagenet) para clasificar cada imagen. Se extraen las características de la útima capa convolucional.

Primero hay que convertir las imágenes a un formato soportado por InceptionV3:
* Redimensionar a 299x299 pixeles
* Preprocesar la imagen utilizando el método $preprocess\_input$ para normalizar la imagen de modo que cada pixel se encuentre en un rango de -1 a 1. 

In [0]:
# Función para preparar las imágenes para InceptionV3

def load_image(image_path):
    img = tf.io.read_file(image_path)
    img = tf.image.decode_jpeg(img, channels=3)
    img = tf.image.resize(img, (299, 299))
    img = tf.keras.applications.inception_v3.preprocess_input(img)
    return img, image_path

## Inicializar InceptionV3 y cargar los pesos preentrenador de Imagenet
Se creará un modelo de Tensorflow Keras, la última capa de este modelo será la última capa convolucional de la arquitectura InceptionV3, esta capa tiene un tamaño de 8x8x2048.

Se utiliza la última capa convolucional para extraer las características. 

* Se pasará cada imagen a través de la red y se guardará el vector resultante en un diccionario de la forma: (image_name : feature_vector)

In [0]:
image_model = tf.keras.applications.InceptionV3(include_top=False,
                                                weights='imagenet')
new_input = image_model.input
hidden_layer = image_model.layers[-1].output

image_features_extract_model = tf.keras.Model(new_input, hidden_layer)

image_features_extract_model.summary()

# Guardando en cach las características extraídas de InceptionV3

Se preprocesará cada imagen con InceptionV3 y se guardarán los resultados como cache en disco. Es posible guardar los resultados en RAM y sería más rápido, pero debido a las limitaciones en cantidad de RAM, se debe realizar el cache en disco.

In [0]:
# Se instalará y preparará tqdm para visualizar el progreso
!pip install tqdm
from tqdm import tqdm

# Get unique images
encode_train = sorted(set(img_name_vector))

# Feel free to change batch_size according to your system configuration
image_dataset = tf.data.Dataset.from_tensor_slices(encode_train)

image_dataset = image_dataset.map(
  load_image, num_parallel_calls=tf.data.experimental.AUTOTUNE).batch(16)

for img, path in tqdm(image_dataset):
  batch_features = image_features_extract_model(img)
  batch_features = tf.reshape(batch_features,
                              (batch_features.shape[0], -1, batch_features.shape[3]))

  for bf, p in zip(batch_features, path):
    path_of_feature = p.numpy().decode("utf-8")
    np.save(path_of_feature, bf.numpy())

## Preprocesando y tokenizando las descripciones


Se tokenizan las descripciones separando por espacios, así obtendremos el vocabulario de todas las palabras únicas en los datos.

In [0]:
# Se busca el tamaño máximo de las descripciones en el dataset
def calc_max_length(tensor):
    return max(len(t) for t in tensor)

Se limitará el tamaño del vocabulario a 10000 palabras para utilizar menos memoria. Se reemplazarán las palabras faltantes con el token "UKN"

In [0]:
# Se eligen las primeras 10000 words from the vocabulary
top_k = 10000
tokenizer = tf.keras.preprocessing.text.Tokenizer(num_words=top_k,
                                                  oov_token="<unk>",
                                                  filters='!"#$%&()*+.,-/:;=?@[\]^_`{|}~ ')
tokenizer.fit_on_texts(train_captions)
train_seqs = tokenizer.texts_to_sequences(train_captions)

tokenizer.word_index['<pad>'] = 0
tokenizer.index_word[0] = '<pad>'

Se crea un mapeo [palabra-indice] y un mapeo [indice-palabra]

In [0]:
# Se generan los vectores de tokens
train_seqs = tokenizer.texts_to_sequences(train_captions)

# Se rellena cada vector con el tamaño máximo de las descripciones presentes
cap_vector = tf.keras.preprocessing.sequence.pad_sequences(train_seqs, padding='post')

# Se calcula max_lenght, un valor que se utilzará para guardar los pesos de la atención
# Calculates the max_length, which is used to store the attention weights
# Esto es, la descripción más larga (en tokens)
max_length = calc_max_length(train_seqs)

## Se separan los datos en TRAIN y TEST

In [0]:
# Se crean los sets de entrenamiento y validación en relación 80-20
img_name_train, img_name_val, cap_train, cap_val = train_test_split(img_name_vector,
                                                                    cap_vector,
                                                                    test_size=0.2,
                                                                    random_state=0)

# imgs de entrenamiento - captions de entrenamiento - imgs de validacioon - captions de validación
len(img_name_train), len(cap_train), len(img_name_val), len(cap_val)

## Se crea el dataset tf.data para el entrenamiento

Las imágenes y las descripciones están listas. Ahora se procederá a crear el dataset de tipo tf.data para entrenar el modelo.

In [0]:
# Función para cargar los archivos de numpy
def map_func(img_name, cap):
  img_tensor = np.load(img_name.decode('utf-8')+'.npy')
  return img_tensor, cap

BATCH_SIZE = 64
BUFFER_SIZE = 1000
embedding_dim = 256
units = 512
vocab_size = top_k + 1
num_steps = len(img_name_train) // BATCH_SIZE
# La dimensión del vector extraído de InceptionV3 es (64, 2048)
# Las dimensiones anteriores se expresan en las variables siguientes
features_shape = 2048
attention_features_shape = 64

dataset = tf.data.Dataset.from_tensor_slices((img_name_train, cap_train))

# Use map to load the numpy files in parallel
dataset = dataset.map(lambda item1, item2: tf.numpy_function(
          map_func, [item1, item2], [tf.float32, tf.int32]),
          num_parallel_calls=tf.data.experimental.AUTOTUNE)

# Shuffle and batch
dataset = dataset.shuffle(BUFFER_SIZE).batch(BATCH_SIZE)
dataset = dataset.prefetch(buffer_size=tf.data.experimental.AUTOTUNE)

## Generando el modelo

El modelo se generará con un decodificador con la misma arquitectura al utilizado para traducción, disponible en el siguiente [paper](https://arxiv.org/pdf/1502.03044.pdf).

La arquitectura está conformada por:
* Las características extraidas de la última capa convolucional de InceptionV3, resultante en un vector con  dimensión (8,8,2048).
* Se redimensionó a una dimensión de (64,2048).
* Este vector se parará a través de un codificador convolucional (que consiste en una capa completamente conectada).
* La red neuronal recurrente (GRU) atenderá las zonas de la imagen para generar la siguiente palabra según el foco de ATENCIÓN

In [0]:
# Módulo de atención
class BahdanauAttention(tf.keras.Model):
  def __init__(self, units):
    super(BahdanauAttention, self).__init__()
    self.W1 = tf.keras.layers.Dense(units)
    self.W2 = tf.keras.layers.Dense(units)
    self.V = tf.keras.layers.Dense(1)

  def call(self, features, hidden):
    # Características de salida del codificador convolucional
    # dimension == (batch_size, 64, embedding_dim)

    # hidden dimension == (batch_size, hidden_size)
    # hidden_with_time_axis shape == (batch_size, 1, hidden_size)
    hidden_with_time_axis = tf.expand_dims(hidden, 1)

    # score dimension == (batch_size, 64, hidden_size)
    score = tf.nn.tanh(self.W1(features) + self.W2(hidden_with_time_axis))

    # attention_weights dimension == (batch_size, 64, 1)
    # Se obtiene un 1 en el último eje porque se aplicó el score a self.V
    attention_weights = tf.nn.softmax(self.V(score), axis=1)

    # context_vector shape after sum == (batch_size, hidden_size)
    context_vector = attention_weights * features
    context_vector = tf.reduce_sum(context_vector, axis=1)

    return context_vector, attention_weights

# Codificador convolucional
class CNN_Encoder(tf.keras.Model):
    # Como las características ya se extrajeron y obtuvieron con pickle
    # El codificador pasa esas características a través de una capa completamente conectada
    def __init__(self, embedding_dim):
        super(CNN_Encoder, self).__init__()
        # shape after fc == (batch_size, 64, embedding_dim)
        self.fc = tf.keras.layers.Dense(embedding_dim)

    def call(self, x):
        x = self.fc(x)
        x = tf.nn.relu(x)
        return x

# Decodificador recurrente
class RNN_Decoder(tf.keras.Model):
  def __init__(self, embedding_dim, units, vocab_size):
    super(RNN_Decoder, self).__init__()
    self.units = units

    self.embedding = tf.keras.layers.Embedding(vocab_size, embedding_dim)
    self.gru = tf.keras.layers.GRU(self.units,
                                   return_sequences=True,
                                   return_state=True,
                                   recurrent_initializer='glorot_uniform')
    self.fc1 = tf.keras.layers.Dense(self.units)
    self.fc2 = tf.keras.layers.Dense(vocab_size)

    self.attention = BahdanauAttention(self.units)

  def call(self, x, features, hidden):
    # Se define la atención como un modelo separado
    context_vector, attention_weights = self.attention(features, hidden)

    # Dimension de x despúes de pasar por embedding == (batch_size, 1, embedding_dim)
    x = self.embedding(x)

    # Dimension de x después de concatenar == (batch_size, 1, embedding_dim + hidden_size)
    x = tf.concat([tf.expand_dims(context_vector, 1), x], axis=-1)

    # Se pasa el vector concatenado a la capa GRU
    output, state = self.gru(x)

    # dimension == (batch_size, max_length, hidden_size)
    x = self.fc1(output)

    # dimension de x == (batch_size * max_length, hidden_size)
    x = tf.reshape(x, (-1, x.shape[2]))

    # dimension de salida == (batch_size * max_length, vocab)
    x = self.fc2(x)

    return x, state, attention_weights

  def reset_state(self, batch_size):
    return tf.zeros((batch_size, self.units))

Se genera el modelo a partir de las clases y métodos generados anteriormente.

In [0]:
encoder = CNN_Encoder(embedding_dim)
decoder = RNN_Decoder(embedding_dim, units, vocab_size)

optimizer = tf.keras.optimizers.Adam()
loss_object = tf.keras.losses.SparseCategoricalCrossentropy(
    from_logits=True, reduction='none')

def loss_function(real, pred):
  mask = tf.math.logical_not(tf.math.equal(real, 0))
  loss_ = loss_object(real, pred)

  mask = tf.cast(mask, dtype=loss_.dtype)
  loss_ *= mask

  return tf.reduce_mean(loss_)

## Checkpoint

In [0]:
checkpoint_path = "./checkpoints/train"
ckpt = tf.train.Checkpoint(encoder=encoder,
                           decoder=decoder,
                           optimizer = optimizer)
ckpt_manager = tf.train.CheckpointManager(ckpt, checkpoint_path, max_to_keep=5)

start_epoch = 0
if ckpt_manager.latest_checkpoint:
  start_epoch = int(ckpt_manager.latest_checkpoint.split('-')[-1])
  # restoring the latest checkpoint in checkpoint_path
  ckpt.restore(ckpt_manager.latest_checkpoint)

## Entrenando el modelo
1. Se extrajeron las características guardadas en los archivos .npy y se pasaron a través del codificador.
2. La salida del codificador, el estado oculto inicializado en 0 y la entrada del decodificador (que es el token inicial) se pasan al decodificador.
3. El decodificador retorna las predicciones y estado oculto del decodificador.
4. El estado oculto del decodificador se pasa de nuevo al modelo para realizar las predicciones y calcular la pérdida.
5. Se utiliza el "teacher forcing" para decidir la siguiente entrada al decodificador.
* Teacher Forcing: Una técnica en donde la palabra objetivo se pasa como la siguiente entrada al decodificador.
6. El último paso es calcular los gradientes, aplicarlos en el optimizador y retropropagar los errores.

In [0]:
# Función que ejecuta un paso de entrenamiento
@tf.function
def train_step(img_tensor, target):
  loss = 0

  # initializing the hidden state for each batch
  # because the captions are not related from image to image
  hidden = decoder.reset_state(batch_size=target.shape[0])

  dec_input = tf.expand_dims([tokenizer.word_index['<start>']] * target.shape[0], 1)

  with tf.GradientTape() as tape:
      features = encoder(img_tensor)

      for i in range(1, target.shape[1]):
          # passing the features through the decoder
          predictions, hidden, _ = decoder(dec_input, features, hidden)

          loss += loss_function(target[:, i], predictions)

          # using teacher forcing
          dec_input = tf.expand_dims(target[:, i], 1)

  total_loss = (loss / int(target.shape[1]))

  trainable_variables = encoder.trainable_variables + decoder.trainable_variables

  gradients = tape.gradient(loss, trainable_variables)

  optimizer.apply_gradients(zip(gradients, trainable_variables))

  return loss, total_loss

### Se comienza el entrenamiento:

In [0]:
# Se añade la lista de loss_plot por separado para evitar que se resetee
# si se vyekve a ejecutar la celda de entrenamiento
loss_plot = []

In [0]:
EPOCHS = 20

for epoch in range(start_epoch, EPOCHS):
    start = time.time()
    total_loss = 0

    for (batch, (img_tensor, target)) in enumerate(dataset):
        batch_loss, t_loss = train_step(img_tensor, target)
        total_loss += t_loss

        if batch % 100 == 0:
            print ('Epoch {} Batch {} Loss {:.4f}'.format(
              epoch + 1, batch, batch_loss.numpy() / int(target.shape[1])))
    # storing the epoch end loss value to plot later
    loss_plot.append(total_loss / num_steps)

    if epoch % 5 == 0:
      ckpt_manager.save()

    print ('Epoch {} Loss {:.6f}'.format(epoch + 1,
                                         total_loss/num_steps))
    print ('Time taken for 1 epoch {} sec\n'.format(time.time() - start))

Se visualizan los resultados del entrenamiento en la siguiente celda.

In [0]:
plt.plot(loss_plot)
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.title('Loss Plot')
plt.show()

## Evaluación del modelo

* La evaluación del modelo es similar a la etapa de entrenamiento, con la excepción que aquí no se utilizará la técnica de "teacher forcing". La entrada del decodificador en cada paso serán sus predicciones anteriores junto con el estado oculto y la salida del codificador.
* Se detiene la predicción cuando el modelo pedice el token final.
* Se almacenan los pesos de atención en cada paso.

In [0]:
# Función que evalúa la predicción de una descripción para una imagen.
def evaluate(image):
    attention_plot = np.zeros((max_length, attention_features_shape))

    hidden = decoder.reset_state(batch_size=1)

    temp_input = tf.expand_dims(load_image(image)[0], 0)
    img_tensor_val = image_features_extract_model(temp_input)
    img_tensor_val = tf.reshape(img_tensor_val, (img_tensor_val.shape[0], -1, img_tensor_val.shape[3]))

    features = encoder(img_tensor_val)

    dec_input = tf.expand_dims([tokenizer.word_index['<start>']], 0)
    result = []

    for i in range(max_length):
        predictions, hidden, attention_weights = decoder(dec_input, features, hidden)

        attention_plot[i] = tf.reshape(attention_weights, (-1, )).numpy()

        predicted_id = tf.random.categorical(predictions, 1)[0][0].numpy()
        result.append(tokenizer.index_word[predicted_id])

        if tokenizer.index_word[predicted_id] == '<end>':
            return result, attention_plot

        dec_input = tf.expand_dims([predicted_id], 0)

    attention_plot = attention_plot[:len(result), :]
    return result, attention_plot

# Función que imprime el plot de atención que se puso a una determinada imagen.
def plot_attention(image, result, attention_plot):
    temp_image = np.array(Image.open(image))

    fig = plt.figure(figsize=(10, 10))

    len_result = len(result)
    for l in range(len_result):
        temp_att = np.resize(attention_plot[l], (8, 8))
        ax = fig.add_subplot(len_result//2, len_result//2, l+1)
        ax.set_title(result[l])
        img = ax.imshow(temp_image)
        ax.imshow(temp_att, cmap='gray', alpha=0.6, extent=img.get_extent())

    plt.tight_layout()
    plt.show()

In [0]:
# Se evalúa el modelo con una imagen aleatoria y sus predicciones
rid = np.random.randint(0, len(img_name_val))
image = img_name_val[rid]

real_caption = ' '.join([tokenizer.index_word[i] for i in cap_val[rid] if i not in [0]])
result, attention_plot = evaluate(image)

print ('Real Caption:', real_caption)
print ('Prediction Caption:', ' '.join(result))
plot_attention(image, result, attention_plot)

## Evaluación con BLEU

In [0]:
Generando listas de descripciones esperadas y las generadas.

In [0]:
real_captions = []
generated_captions = []
img_path = []

n_tests = 300

for i,image in enumerate(img_name_val[:n_tests]):
  real_captions.append(' '.join([tokenizer.index_word[i] for i in cap_val[i] if i not in [0]]))
  caption, attention_plot = evaluate(image)
  img_path.append(image)
  generated_captions.append(caption)
  if i % 100 == 0: print("Describiendo id %d" %(i)) 

print("Finalizado...")

In [0]:
import warnings
warnings.filterwarnings('ignore')

from nltk.translate.bleu_score import sentence_bleu

import seaborn as sns

tokenized_real_captions = []
tokenized_generated_captions = []
# Se tokenizan las descripciones reales y se eliminan las etiquetas de
# <start> <end>
for caption in generated_captions:
  d_caption = caption[:len(caption)-1]
  tokenized_generated_captions.append(d_caption)

for caption in real_captions:
  back_caption = caption
  back_caption = back_caption.split()
  back_caption = back_caption[1:-1]
  tokenized_real_captions.append(back_caption)


# Se separarán las buenas predicciones de las malas.
pred_good, pred_bad, bleus, meteors = [], [], [], []
for i, path in enumerate(img_path):
  caption_true = tokenized_real_captions[i]
  gen_caption = tokenized_generated_captions[i]
  bleu = sentence_bleu([caption_true], gen_caption)
  bleus.append(bleu)
  if bleu > 0.7:
    pred_good.append((bleu,path,caption_true,gen_caption))
  elif bleu < 0.3:
    pred_bad.append((bleu,path,caption_true,gen_caption))

# Visualizando la distribución de los BLEUs
sns.set(color_codes = True)
sns.distplot(bleus)

print("\t\t\t BLEU")
print("BLEU más alto: ", np.amax(bleus))
print("BLEU más bajo: ", np.amin(bleus))
print("Puntuación BLEU media {:4.3f}".format(np.mean(bleus)))
print("Puntuación BLEU mediana {:4.3f}".format(np.median(bleus)))
print("Desviación estándar de las puntuaciones {:4.3f}".format(np.std(bleus)))

Se visualizan las imágenes con las predicciones buenas.

In [0]:
def plot_images(generated_captions):
    def create_str(caption_true):
        strue = ""
        for s in caption_true:
            strue += " " + s
        return(strue)
    npix = 224
    target_size = (npix,npix,3)    
    count = 1
    fig = plt.figure(figsize=(10,20))
    npic = len(generated_captions)
    for pb in generated_captions:
        bleu,jpgfnm,caption_true,caption = pb
        ## images 
        filename = jpgfnm
        image_load = load_img(filename, target_size=target_size)
        ax = fig.add_subplot(npic,2,count,xticks=[],yticks=[])
        ax.imshow(image_load)
        count += 1

        caption_true = create_str(caption_true)
        caption = create_str(caption)
        
        ax = fig.add_subplot(npic,2,count)
        plt.axis('off')
        ax.plot()
        ax.set_xlim(0,1)
        ax.set_ylim(0,1)
        ax.text(0,0.7,"real:" + caption_true,fontsize=20)
        ax.text(0,0.4,"generada:" + caption,fontsize=20)
        ax.text(0,0.1,"BLEU: {}".format(bleu),fontsize=20)
        count += 1
    plt.show()

print("Descripciones malas")
plot_images(pred_bad[:5])
print("Descripciones buenas")
plot_images(pred_good[:5])